In [1]:
import sys
sys.path.append("..")

<module 'src.utils.linearization_utils' from '/Users/josifosk/Documents/PhD/SynthIE_main/notebooks/../src/utils/linearization_utils.py'>

In [66]:
class_name_e  = "fully_expanded"
lc_e = src.utils.linearization_utils.get_linearization_class(class_name_e)

class_name_et  = "fully_expanded_et"
lc_et = src.utils.linearization_utils.get_linearization_class(class_name_et)

In [19]:
import os
import json
import numpy as np
import pandas as pd

DATA_DIR='../data'

def read_jsonl(path_to_file):
    with open(path_to_file, 'r') as f:
        data = [json.loads(line) for line in f]
    return data

def get_data(data_dir, split):
    data = read_jsonl(os.path.join(data_dir, split + '.jsonl'))
    return data

In [67]:
dataset_name='sdg'
split='val_code_davinci'
data = get_data(os.path.join(DATA_DIR, dataset_name), split)

In [68]:
triplets_text = IEGenericDataset.get_triplets_surface_form(data[0])

In [73]:
lc_e.triplet_list_to_text(triplets_text)

('[s] Swedish Open Cultural Heritage [r] file_format [o] XML [e] [s] Swedish Open Cultural Heritage [r] part_of [o] Semantic Web [e] [s] Swedish Open Cultural Heritage [r] programming_language [o] Java (programming language) [e] [s] Swedish Open Cultural Heritage [r] instance_of [o] Free software [e]',
 None)

In [72]:
text, ids = lc_et.triplet_list_to_text(triplets_text, tokenizer)

In [76]:
len(ids)

96

In [46]:
import re

surface_forms = {
    "{subject_id}": "[s]",
    "{relation_id}": "[r]",
    "{object_id}": "[o]",
    "{et_id}": "[et]",
    "{separator}": " "
  }

regex = re.compile("|".join([re.escape(surface_forms[_id])
                                for _id in ["{subject_id}", "{relation_id}", "{object_id}", "{et_id}"]]))

In [71]:
text_parts = [
    element.strip(surface_forms['{separator}']).strip().strip(surface_forms['{separator}'])
    for element in re.split(regex, text) if element.strip()
]

len(text_parts)

12

In [40]:
lc_et.text_to_triplet_list(text[:-50])

{('Swedish Open Cultural Heritage', 'file_format', 'XML'),
 ('Swedish Open Cultural Heritage', 'part_of', 'Semantic Web'),
 ('Swedish Open Cultural Heritage',
  'programming_language',
  'Java (programming language)')}

In [11]:
import json
import numpy as np
import os 
import jsonlines
from tqdm import tqdm
from transformers import T5Tokenizer


tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-base")
target_linearization = {
  "formatted_triplet": ["{subject_id}", " ", "{subject}", " ", "{relation_id}", " ", "{relation}", " ", "{object_id}", " ", "{object}", " ", "{et_id}"],
  "keep_spaces": False,
  "surface_forms": {
    "{subject_id}": "[s]",
    "{relation_id}": "[r]",
    "{object_id}": "[o]",
    "{et_id}": "[et]",
    "{separator}": " "
  }
}         

def _get_triplet_surfaceform(triplet):
    return triplet["subject"]["surfaceform"], triplet["predicate"]["surfaceform"], triplet["object"]["surfaceform"]

In [1]:
import re

In [15]:
text = "[s] sub1 [r] rel1 [o] obj 1 [et] [s] sub2 [r] rel2 [o] obj 2 [et] [s] sub3 [r] rel3 [o] obj3 [et]"
text_parts = [
            element.strip() for element in re.split(re.compile("\[s\]|\[r\]|\[o\]|\[et\]|\[e\]"), text) if element.strip()
        ]

In [16]:
surface_forms = {
    "{subject_id}": "[s]",
    "{relation_id}": "[r]",
    "{object_id}": "[o]",
    "{et_id}": "[et]",
    "{separator}": " "
}

In [17]:
text_parts

['sub1', 'rel1', 'obj 1', 'sub2', 'rel2', 'obj 2', 'sub3', 'rel3', 'obj3']

In [24]:
text_parts2 = [
            element.strip(surface_forms['{separator}']).strip().strip(surface_forms['{separator}']) for element in re.split(re.compile("|".join([re.escape(surface_forms[_id]) for _id in ["{subject_id}", "{relation_id}", "{object_id}", "{et_id}"]])), text) if element.strip()
        ]
text_parts2

['sub1', 'rel1', 'obj 1', 'sub2', 'rel2', 'obj 2', 'sub3', 'rel3', 'obj3']

In [2]:
re.escape("[s]")

'\\[s\\]'

In [45]:
def process_split(split, output_split, folder):
    path_to_file = os.path.join(folder, split + '.jsonl')
    data = read_jsonl(path_to_file)

    for obj in tqdm(data):
        triplets_list = [_get_triplet_surfaceform(triplet) for triplet in obj["triplets"]]
        _, ids = TripletUtils.linearize_triplet_list_complex(triplets_list, tokenizer=tokenizer, **target_linearization)
        obj['num_tokens_target'] = len(ids)
        obj['num_tokens_input'] = len(tokenizer.encode(obj['text'].strip()))

    output_file_path = os.path.join(folder, output_split + '.jsonl')
    with jsonlines.open(output_file_path, "w") as writer:
        writer.write_all(data)

### Add Target Linearization and length of input and target

#### Rebel

In [66]:
folder = '../data/rebel'

split="train" 
output_split = "train"

process_split(split, output_split, folder)

split="val" 
output_split = "val"

process_split(split, output_split, folder)

split="test" 
output_split = "test"

process_split(split, output_split, folder)

100%|██████████| 993847/993847 [14:09<00:00, 1170.52it/s]


##### SDG (code_davinci)

In [14]:
folder = '../data/sdg'
split = "_train_code_davinci.jsonl"
process_split(split, split, folder)

folder = '../data/sdg'
split = "test_code_davinci"
process_split(split, split, folder)

100%|██████████| 179255/179255 [06:14<00:00, 478.93it/s]


### Split SDG train on train and val

In [9]:
def split_on_train_val(data, num_val_datapoints, identifier, seed=123):
    random_state = np.random.RandomState(seed)

    val_indices = random_state.choice(len(data), num_val_datapoints, replace=False)
    train_indices = np.setdiff1d(np.arange(len(data)), val_indices)

    assert len(np.intersect1d(train_indices, val_indices)) == 0
    assert len(np.union1d(train_indices, val_indices)) == len(data)

    val_data = [data[i] for i in sorted(val_indices)]
    output_file_path = os.path.join(folder, f"val_{identifier}.jsonl")
    with jsonlines.open(output_file_path, "w") as writer:
        writer.write_all(val_data)

    train_data = [data[i] for i in sorted(train_indices)]
    output_file_path = os.path.join(folder, f"train_{identifier}.jsonl")
    with jsonlines.open(output_file_path, "w") as writer:
        writer.write_all(train_data)

folder = '../data/sdg'
split = "_train_code_davinci"
path_to_file = os.path.join(folder, split + '.jsonl')
data = read_jsonl(path_to_file)

split_on_train_val(data, 15000, "code_davinci")

### Subsample the REBEL test set

In [49]:
def subsample(folder, split, num_datapoints, identifier, seed=12345):
    path_to_file = os.path.join(folder, split + '.jsonl')
    data = read_jsonl(path_to_file)

    random_state = np.random.RandomState(seed)

    indices = random_state.choice(len(data), num_datapoints, replace=False)
    data = [data[i] for i in sorted(indices)]
    
    output_file_path = os.path.join(folder, f"{split}_{identifier}.jsonl")
    print(f"Writing {len(data)} datapoints to {output_file_path}")
    with jsonlines.open(output_file_path, "w") as writer:
        writer.write_all(data)

folder = '../data/rebel'
split =  'test'
_id2num_datapoints = {"20": 20, "10k": 10000, "20k": 20000}

for _id, num_datapoints in _id2num_datapoints.items():
    subsample(folder, split, num_datapoints, _id)

Writing 20 datapoints to ../data/rebel/test_20.jsonl
Writing 10000 datapoints to ../data/rebel/test_10k.jsonl
Writing 20000 datapoints to ../data/rebel/test_20k.jsonl


### Subsample the SDG test set

In [11]:
def subsample(folder, split, num_datapoints, identifier, seed=12345):
    path_to_file = os.path.join(folder, split + '.jsonl')
    data = read_jsonl(path_to_file)

    random_state = np.random.RandomState(seed)

    indices = random_state.choice(len(data), num_datapoints, replace=False)
    data = [data[i] for i in sorted(indices)]
    
    output_file_path = os.path.join(folder, f"{split}_{identifier}.jsonl")
    print(f"Writing {len(data)} datapoints to {output_file_path}")
    with jsonlines.open(output_file_path, "w") as writer:
        writer.write_all(data)

folder = '../data/sdg'
split =  'test_code_davinci'
_id2num_datapoints = {"20": 20, "10k": 10000} #,  "20k": 20000}

for _id, num_datapoints in _id2num_datapoints.items():
    subsample(folder, split, num_datapoints, _id)

Writing 20 datapoints to ../data/sdg/test_code_davinci_20.jsonl
Writing 10000 datapoints to ../data/sdg/test_code_davinci_10k.jsonl


### Old dev code

In [19]:
n_target_tokens = [dp["num_tokens_target"] for dp in data]
n_input_tokens = [dp["num_tokens_input"] for dp in data]

In [23]:
max(n_target_tokens), min(n_target_tokens)

(523, 19)

In [30]:
import numpy as np 
import matplotlib.pyplot as plt
%matplotlib inline

def visualize(num_tokens, binwidth=10):
    plt.hist(num_tokens, bins=np.arange(min(num_tokens), max(num_tokens) + binwidth, binwidth))
    plt.title(f"[{label}] Token distribution for {field}")
    plt.show()



In [24]:
def visualize_token_distribution(dataset, label, field="text"):
    binwidth = 10
    num_tokens = [datamodule.tokenizer(dp[field], return_tensors="pt")['input_ids'].numel() for dp in dataset]
    # plt.hist(num_tokens, bins=np.arange(min(num_tokens), max(num_tokens) + binwidth, binwidth))
    # plt.title(f"[{label}] Token distribution for {field}")
    # plt.show()
    print(f"[{label}, {field}] Max {max(num_tokens)}, Min {min(num_tokens)}")

In [25]:
visualize_token_distribution(data, "train_code_davinci", field="text")

NameError: name 'datamodule' is not defined

In [11]:
from typing import List, Tuple, Union


class TripletUtils(object):
    @staticmethod
    def normalize_spaces(name, keep_spaces: bool) -> str:
        name = name.strip()
        if keep_spaces:
            return name.replace("_", " ")
        
        return name.replace(" ", "_")
        

    @staticmethod
    def linearize_triplet_list(
        triplet_list: List[Tuple[str, str, str]], formatted_triplet: str, separator: str, keep_spaces: bool, tokenizer=None
    ) -> str:
        surface_forms = {"{}": sub, "{relation}": rel, "{object}": obj, "{sep}": separator}
        linearized_triplets = []

        for sub, relation, object in triplet_list:
            surface_form = {"{subject}": sub, "{relation}": rel, "{object}": obj, "{sep}": separator}

            sub = TripletUtils.normalize_spaces(surface_form["{subject}"], keep_spaces)
            rel = TripletUtils.normalize_spaces(surface_form["{relation}"], True)
            obj = TripletUtils.normalize_spaces(surface_form["{object}"], keep_spaces)

            linearized_triplets.append(
                formatted_triplet.replace("{subject}", sub).replace("{relation}", rel).replace("{object}", obj)
            )

        return separator.join(linearized_triplets)


    @staticmethod
    def linearize_triplet_list_complex(triplet_list: List[Tuple[str, str, str]], formatted_triplet: list, surface_forms, keep_spaces: bool, tokenizer=None):
        linearized_triplets = []

        for sub, rel, obj in triplet_list:
            surface_forms.update({"{subject}": sub, "{relation}": rel, "{object}": obj})

            surface_forms["{subject}"] = TripletUtils.normalize_spaces(surface_forms["{subject}"], keep_spaces)
            surface_forms["{relation}"] = TripletUtils.normalize_spaces(surface_forms["{relation}"], True)
            surface_forms["{object}"] = TripletUtils.normalize_spaces(surface_forms["{object}"], keep_spaces)

            linearized_triplets.extend([surface_forms.get(item, item) for item in formatted_triplet])
            linearized_triplets.append(surface_forms['{separator}'])
           
        linearized_triplets = linearized_triplets[:-1]
        linearized_triplet = "".join(linearized_triplets)

        if tokenizer is None:
            return linearized_triplet, None

        tokenized_linearized_triplet_parts = tokenizer(linearized_triplets, return_attention_mask=False)['input_ids']
        tokenized_linearized_triplet = [token for part in tokenized_linearized_triplet_parts for token in part[:-1]] + [tokenizer.eos_token_id]
        return linearized_triplet, tokenized_linearized_triplet

In [61]:
text1, input_ids1 = TripletUtils.linearize_triplet_list_complex([("E1", "R1", "E2"), ("E3", "R2_2", "E4 3")], tokenizer=tokenizer, **target_linearization)
text2, input_ids2 = TripletUtils.linearize_triplet_list_complex([("E1", "R1", "E2"), ("E3", "R2", "E4"), ("E5", "R3", "R6")], tokenizer=tokenizer, **target_linearization)
